<a href="https://colab.research.google.com/github/Doppler-67/car-price-prediction/blob/main/autos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing libraries**

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
sns.set()

# **Data understanding**

**План исследования:**


1.   Загрузить данные.
2.   Провести предобработку данных.
3.   Провести исследовательский анализ данных.
4.   Протестировать различные модели.
5.   Подобрать оптимальные параметры для моделей.
6.   Выбрать лучшую модель и проверить ее на тестовых данных.
7.   Провести интерпретацию лучшей модели.
8.   Сделать вывод о результатах.



In [3]:
data = pd.read_csv('/content/drive/MyDrive/autos_ds_project/autos.csv')

In [4]:
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [6]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


Нам доступно 354369 строк данных. Приступим в предобработке данных.

# **Data preparation**

Для начала исправим названия колонок (уберем заглавные символы и применим змеиный регистр).

In [7]:
data.columns = ['date_crawled', 'price', 'vehicle_type', 'registration_year', 
                'gearbox','power', 'model', 'kilometer', 'registration_month',
                'fuel_type', 'brand', 'is_repaired', 'date_created', 
                'number_of_pictures', 'postal_code', 'last_seen']

Теперь подправим типы данных.

In [8]:
dt_cols = ['date_crawled', 'date_created', 'last_seen'] 
data[dt_cols] = data[dt_cols].apply(pd.to_datetime)

In [9]:
int_cols = ['price', 'registration_year', 'power', 'kilometer', 'registration_month', 'number_of_pictures', 'postal_code']
data[int_cols] = data[int_cols].astype('int32')

In [10]:
data['is_repaired'] = data['is_repaired'].fillna('unknown')

В колонке is_repaired присуствует много пропусков. Так как эта колонка содержит всего две категории ( yes и no), то присвоим пропускам категорию - 'unknown'.

Проанализируем колонку number_of_pictures.

In [11]:
data['number_of_pictures'].unique()

array([0], dtype=int32)

Как мы видим в колонке number_of_pictures все значения равны 0. Уадлим ее, так как данная колонка бесполезна.

In [12]:
data = data.drop('number_of_pictures', axis = 1)

Изучим колонку date_created.




In [13]:
data['date_created'].dt.year.value_counts()

2016    354343
2015        25
2014         1
Name: date_created, dtype: int64

В датасете в основном анкеты были созданы в 2016 году.

Нам не понадобятся колонки date_crawled, date_created, last_seen, так как они никакой полезной информации в себе не несут. Поэтому удалим их.

In [14]:
data = data.drop(['date_crawled', 'date_created', 'last_seen'], axis = 1)

Изучим колонку vehicle_type.

In [15]:
data['vehicle_type'].value_counts()

sedan          91457
small          79831
wagon          65166
bus            28775
convertible    20203
coupe          16163
suv            11996
other           3288
Name: vehicle_type, dtype: int64

In [16]:
data['vehicle_type'].isna().sum()

37490

Пропусков много. Поступим с ними также, как в колонке is_reapired.

In [17]:
data['vehicle_type'] = data['vehicle_type'].fillna('unknown')

В данных присутсвуют строки в которых цена машины равна нулю. Такого быть не может, поэтому удалим их.

In [18]:
len(data[data['price'] == 0])

10772

In [19]:
data = data[data['price'] != 0]

В колонке power тоже много нулевых значений. Можно попробовать как-нибудь спарсить значение лошадиных сил по имеющейся марке и модели автомобиля. Такой вариант потребует много времени и усилий. Попробуем что-то попробще. Так как это не категориальная колонка, то мы не можем просто создать новую категорию 'unknown', поэтому просто удалим эти строки.

In [20]:
len(data[data['power'] == 0])

36360

In [21]:
data = data[data['power'] != 0]

Изучим  колонку fuel_type.

In [22]:
data['fuel_type'].value_counts()

petrol      192148
gasoline     89582
lpg           4798
cng            501
hybrid         205
other           97
electric        77
Name: fuel_type, dtype: int64

In [23]:
data['fuel_type'].isna().sum()

19829

Также много пропусков. Заполним их значением 'unknown'.

In [24]:
data['fuel_type'] = data['fuel_type'].fillna('unknown')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Изучим колонку model.

In [25]:
data['model'].value_counts()

golf                  26031
other                 20829
3er                   17795
polo                  11157
corsa                 10568
                      ...  
i3                        5
samara                    5
rangerover                3
serie_3                   3
range_rover_evoque        2
Name: model, Length: 249, dtype: int64

В данной колонки слишком много категорий. Оставим только 20 самых пополярных, а остальных поместим в категорию 'other'.

In [26]:
top20 = data['model'].value_counts()[:21].index # 21 потому что в этом топе присуствует значение other.

In [27]:
data['model'] = data['model'].apply(lambda x: 'other' if x not in top20 else x)

In [28]:
data['model'].isna().sum()

0

Также мы видим, что в колонке model есть значительное кол-во пропусков. Заполним их значением 'unknown'.

In [29]:
data['model'] = data['model'].fillna('unknown')

Изучим колонку gearbox.

In [30]:
data['gearbox'].value_counts()

manual    240694
auto       60434
Name: gearbox, dtype: int64

In [31]:
data['gearbox'].isna().sum()

6109

В колонке gearbox пропусков незначительное кол-во. Можно заполнить пропуски значением 'auto', чтобы немного уменьшить дисбаланс классов, но давайте не будет искажать данные и просто удалим пропуски.

In [32]:
data = data.dropna(subset=['gearbox'])

Предобработка завершена. Посмотрим на предобработанные данные и перейдем к EDA.

In [33]:
data.head()

,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,is_repaired,postal_code
1,18300,coupe,2011,manual,190,other,125000,5,gasoline,audi,yes,66954
2,9800,suv,2004,auto,163,other,125000,8,gasoline,jeep,unknown,90480
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,91074
4,3600,small,2008,manual,69,other,90000,7,gasoline,skoda,no,60437
5,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,33775


In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 301128 entries, 1 to 354368
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   price               301128 non-null  int32 
 1   vehicle_type        301128 non-null  object
 2   registration_year   301128 non-null  int32 
 3   gearbox             301128 non-null  object
 4   power               301128 non-null  int32 
 5   model               301128 non-null  object
 6   kilometer           301128 non-null  int32 
 7   registration_month  301128 non-null  int32 
 8   fuel_type           301128 non-null  object
 9   brand               301128 non-null  object
 10  is_repaired         301128 non-null  object
 11  postal_code         301128 non-null  int32 
dtypes: int32(6), object(6)
memory usage: 23.0+ MB
